In [1]:
import pandas as pd
import os
import sys

sys.path+=['../']
import param
import gc
import json
import gzip


In [2]:
mergedStatOutDir='/nrnb/users/btsui/Data/all_seq/rnaseq_merged/merged_kallisto_run_info.pickle'

In [3]:
outFnameS=pd.Series(os.listdir(param.count_out_dir))

In [4]:
outFnameS.head()

0    SRR3553383.abundance.tsv.gz
1    SRR5418097.abundance.tsv.gz
2    ERR1172947.run_info.json.gz
3    SRR5308103.abundance.tsv.gz
4    SRR5606337.abundance.tsv.gz
dtype: object

In [5]:
abundanceTsvS=outFnameS[outFnameS.str.contains('.run_info.json.gz$')]

In [6]:
inJsonDirS=param.count_out_dir+'/'+abundanceTsvS

inParseDirS=inJsonDirS#.iloc[:1000]

In [7]:
from multiprocessing import Pool
from tqdm import tqdm

In [8]:
def parseOne ( inJsonDir):
    try:
        with gzip.open(inJsonDir, 'rb') as f:
            json1_data = json.loads(f.read())#[0]
            json1_data['fname']=inJsonDir
            tmpS=pd.Series(json1_data)
    except:
        return None
    return tmpS

In [9]:
with Pool(64) as p:
    myL=list(tqdm(p.imap(parseOne,inParseDirS)
                  ,total=len(inParseDirS)
                 ))

100%|██████████| 454926/454926 [07:58<00:00, 950.69it/s]


In [10]:
myDf=pd.DataFrame(myL)#['call']

In [11]:
mergedStatDf=pd.DataFrame(myL)

In [15]:
mergedStatDf['Run']=mergedStatDf['fname'].str.split('/').str[-1].str.split('.').str[0]

In [16]:
mergedStatDf.to_pickle(mergedStatOutDir)

In [18]:
#